<a href="https://colab.research.google.com/github/bohachu/colab/blob/master/julia_on_colab_pm25_hourly_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Installation cell
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [36]:
import Pkg; Pkg.add("Dates")
import Dates
Dates.now()
if !isfile("epa_micro_20200101.zip")
    run(`wget -q --no-check-certificate https://ci.taiwan.gov.tw/dsp/history/epa_micro/202001/epa_micro_20200101.zip`)
    run(`unzip epa_micro_20200101.zip`)
end

 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


In [37]:
@time begin
    println("$(Dates.now()), add packages")
    import Pkg; Pkg.add(["DataFramesMeta","CSV","DataFrames","Statistics"])
    using DataFramesMeta,CSV,DataFrames,Statistics
end

function go()
    str_file_name="epa_micro_20200101.csv"
    df_pm25 = CSV.File(str_file_name) |> DataFrame!
    df_pm25=dropmissing(df_pm25, Symbol("PM2.5"))
    select!(df_pm25, Not(:humidity))
    select!(df_pm25, Not(:temperature))
    cut_to_hour(str_time)=str_time[1:14]*"00:00"
    df_pm25=hcat(df_pm25, @with(df_pm25, cut_to_hour.(:time)))
    select!(df_pm25, Not(:time))    
    rename!(df_pm25, Symbol.(["id","pm25","hour"]))
    df_pm25=by(df_pm25, [:id,:hour], df -> round(mean(df[:, :pm25]),digits=2))
    rename!(df_pm25, Symbol.(["id","hour","pm25_mean"]))
    df_pm25=sort(df_pm25,:id)
    CSV.write("hourly_$(str_file_name)",df_pm25)
end

@time begin
    println("$(Dates.now()), go() begin")
    go()
    println("$(Dates.now()), go() end")
end

2020-04-25T10:17:28.249, add packages
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
  0.137611 seconds (844.02 k allocations: 46.924 MiB)
2020-04-25T10:17:28.435, go() begin
2020-04-25T10:17:29.855, go() end
  1.420038 seconds (6.73 M allocations: 802.957 MiB, 10.76% gc time)


In [0]:
#wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eIVoNhqBglC4qfSIan6Dpn49bx1hxmLI' -O a.ipynb
#time jupyter nbconvert --ExecutePreprocessor.timeout=-1 --to notebook --execute a.ipynb